# Project 5: Sentiment Analysis with Basic NLP
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment. After implementing the function, run the cell to test it against the unit tests we've provided. For each problem, we provide one or more unit tests from our `project_tests` package. These unit tests won't tell you if your answer is correct, but will warn you of any major errors. Your code will be checked for the correct solution when you submit it to Udacity.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

The other packages that we're importing are `project_helper` and `project_tests`. These are custom packages built to help you solve the problems.  The `project_helper` module contains utility functions and graph functions. The `project_tests` contains the unit tests for all the problems.

### Install Packages

In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt

    100% |████████████████████████████████| 1.4MB 370kB/s eta 0:00:01
    100% |████████████████████████████████| 17.0MB 31kB/s  eta 0:00:01   16% |█████▍                          | 2.9MB 25.2MB/s eta 0:00:01    23% |███████▋                        | 4.0MB 21.1MB/s eta 0:00:01    30% |█████████▊                      | 5.2MB 25.4MB/s eta 0:00:01    51% |████████████████▋               | 8.8MB 18.1MB/s eta 0:00:01    65% |█████████████████████           | 11.1MB 24.9MB/s eta 0:00:01    79% |█████████████████████████▌      | 13.5MB 25.0MB/s eta 0:00:01    86% |███████████████████████████▊    | 14.7MB 25.5MB/s eta 0:00:01    92% |█████████████████████████████▊  | 15.7MB 23.5MB/s eta 0:00:01    98% |███████████████████████████████▌| 16.7MB 10.6MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 4.9MB/s eta 0:00:01
  Running setup.py bdist_wheel for nltk ... done
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
  Running

### Load Packages

In [2]:
import nltk
import numpy as np
import pandas as pd
import pickle
import pprint
import project_helper
import project_tests

from tqdm import tqdm

### Download NLP Corpora
You'll need two corpora to run this project: the stopwords corpus for removing stopwords and wordnet for lemmatizing.

In [3]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

## Get 10ks
We'll be running NLP anlalysis on 10-k documents. To do that, we first need to download the documents. For this project, we'll download 10-ks for Apple and Amazon. To lookup documents for these companies, we'll use their CIK.

In [4]:
cik_lookup = {'AMZN': '0000320193', 'AAPL': '0001018724'}

### Get list of 10-ks
Let's pull a list of filled 10-ks from the SEC for each company.

In [5]:
from bs4 import BeautifulSoup


def get_sec_data(cik, doc_type, start=0, count=10):
    rss_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany' \
        '&CIK={}&type={}&start={}&count={}&owner=exclude&output=atom' \
        .format(cik, doc_type, start, count)
    page = project_helper.call_sec(rss_url)
    feed = BeautifulSoup(page.text.encode('ascii'), 'xml').feed
    entries = [
        (
            entry.content.find('filing-href').getText(),
            entry.content.find('filing-type').getText(),
            entry.content.find('filing-date').getText())
        for entry in feed.find_all('entry', recursive=False)]

    return entries

Let's pull the list using the `get_sec_data` function, then display some of the results. For displaying some of the data, we'll use Amazon as an example. 

In [6]:
example_ticker = 'AMZN'
sec_data = {}

for ticker, cik in cik_lookup.items():
    sec_data[ticker] = get_sec_data(cik, '10-K')

pprint.pprint(sec_data[example_ticker][:5])

[('http://www.sec.gov/Archives/edgar/data/320193/000032019318000145/0000320193-18-000145-index.htm',
  '10-K',
  '2018-11-05'),
 ('http://www.sec.gov/Archives/edgar/data/320193/000032019317000070/0000320193-17-000070-index.htm',
  '10-K',
  '2017-11-03'),
 ('http://www.sec.gov/Archives/edgar/data/320193/000162828016020309/0001628280-16-020309-index.htm',
  '10-K',
  '2016-10-26'),
 ('http://www.sec.gov/Archives/edgar/data/320193/000119312515356351/0001193125-15-356351-index.htm',
  '10-K',
  '2015-10-28'),
 ('http://www.sec.gov/Archives/edgar/data/320193/000119312514383437/0001193125-14-383437-index.htm',
  '10-K',
  '2014-10-27')]


### Download 10-ks
As you see, this is a list of urls. These urls point to a file that contains meta data related to each filling. Since we don't care about the meta data, we'll pull the filling by replacing the url with the filling url.

In [7]:
raw_fillings_by_ticker = {}
for ticker, data in sec_data.items():
    raw_fillings_by_ticker[ticker] = {}
    for index_url, file_type, file_date in tqdm(data, desc='Downloading {} Fillings'.format(ticker), unit='filling'):
        if (file_type == '10-K'):
            file_url = index_url.replace('-index.htm', '.txt').replace('.txtl', '.txt')            
            
            raw_fillings_by_ticker[ticker][file_date] = project_helper.call_sec(file_url).text


print('Example Document:\n\n{}...'.format(next(iter(raw_fillings_by_ticker[example_ticker].values()))[:1000]))

Example Document:

<SEC-DOCUMENT>0000320193-18-000145.txt : 20181105
<SEC-HEADER>0000320193-18-000145.hdr.sgml : 20181105
<ACCEPTANCE-DATETIME>20181105080140
ACCESSION NUMBER:		0000320193-18-000145
CONFORMED SUBMISSION TYPE:	10-K
PUBLIC DOCUMENT COUNT:		88
CONFORMED PERIOD OF REPORT:	20180929
FILED AS OF DATE:		20181105
DATE AS OF CHANGE:		20181105

FILER:

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			APPLE INC
		CENTRAL INDEX KEY:			0000320193
		STANDARD INDUSTRIAL CLASSIFICATION:	ELECTRONIC COMPUTERS [3571]
		IRS NUMBER:				942404110
		STATE OF INCORPORATION:			CA
		FISCAL YEAR END:			0930

	FILING VALUES:
		FORM TYPE:		10-K
		SEC ACT:		1934 Act
		SEC FILE NUMBER:	001-36743
		FILM NUMBER:		181158788

	BUSINESS ADDRESS:	
		STREET 1:		ONE APPLE PARK WAY
		CITY:			CUPERTINO
		STATE:			CA
		ZIP:			95014
		BUSINESS PHONE:		(408) 996-1010

	MAIL ADDRESS:	
		STREET 1:		ONE APPLE PARK WAY
		CITY:			CUPERTINO
		STATE:			CA
		ZIP:			95014

	FORMER COMPANY:	
		FORMER CONFORMED NAME:	APPLE COMPUT

### Get Documents
With theses fillings downloaded, we want to break them into their associated documents. These documents are sectioned off in the fillings with the tags `<DOCUMENT>` for the start of each document and `</DOCUMENT>` for the end of each document. There's no overlap with these documents, so each `</DOCUMENT>` tag should come after the `<DOCUMENT>` with no `<DOCUMENT>` tag inbetween.

Implement `get_documents` to return a list of these documents from a filling. Make sure not to include the tag in the returned document text.

In [ ]:
import re


def get_documents(text):
    """
    Extract the documents from the text

    Parameters
    ----------
    text : str
        The text with the document strings inside

    Returns
    -------
    extracted_docs : list of str
        The document strings found in `text`
    """
    
    # TODO: Implement
    
    return None


project_tests.test_get_documents(get_documents)

With the `get_documents` function implemented, let's extract all the documents.

In [ ]:
filling_documents_by_ticker = {}
for ticker, raw_fillings in raw_fillings_by_ticker.items():
    filling_documents_by_ticker[ticker] = {}
    for file_date, filling in raw_fillings.items():
        filling_documents_by_ticker[ticker][file_date] = get_documents(filling)


print('\n\n'.join([
    'Document {} Filed on {}:\n{}...'.format(doc_i, file_date, doc[:200])
    for file_date, docs in filling_documents_by_ticker[example_ticker].items()
    for doc_i, doc in enumerate(docs)][:3]))

### Get Document Types
Not that we have all the documents, we want to find the 10-k form in this 10-k filling. Implement the `get_document_type` function to return the type of document given. The document type is located on a line with the `<TYPE>` tag. For example, a form of type "TEST" would have the line `<TYPE>TEST`.

In [ ]:
def get_document_type(doc):
    """
    Return the document type lowercased

    Parameters
    ----------
    doc : str
        The document string

    Returns
    -------
    doc_type : str
        The document type lowercased
    """
    
    # TODO: Implement
    
    return None


project_tests.test_get_document_type(get_document_type)

With the `get_document_type` function, we'll filter out all non 10-k documents.

In [ ]:
ten_ks_by_ticker = {}
for ticker, filling_documents in filling_documents_by_ticker.items():
    ten_ks_by_ticker[ticker] = []
    for file_date, documents in filling_documents.items():
        for document in documents:
            if get_document_type(document) == '10-k':
                ten_ks_by_ticker[ticker].append({
                    'cik': cik_lookup[ticker],
                    'file': document,
                    'file_date': file_date})


project_helper.print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['cik', 'file', 'file_date'])

### Get Items
Each 10-k form is broken into items. For this project, we want item 1a, 7, and 7a. We'll use the function `find_items` to extract the items from the document.

In [ ]:
def find_items(ten_k):
    fin_pattern=re.compile(r'(>Item(\s|&#160;|&nbsp;|)[01]{0,1}[0-9][AB]{0,1}\.{0,1})|(ITEM\s[01]{0,1}[0-9][AB]{0,1})')
    fin_match=re.finditer(fin_pattern,(ten_k))
    
    test_df=pd.DataFrame([(x.group(),x.start(),x.end()) for x in (fin_match)])
    test_df.columns=['item','start','end']
    test_df['item']=test_df.item.str.lower()
    test_df.replace('&#160;',' ',regex=True,inplace=True)
    test_df.replace('&nbsp;',' ',regex=True,inplace=True)
    test_df.replace('\.','',regex=True,inplace=True)
    test_df.replace({'item': r'^>'}, {'item': ' '}, regex=True,inplace=True)
    
    pos_dat=test_df.sort_values('start', ascending=True).drop_duplicates(subset=['item'],keep='last')
    pos_dat.replace(' ','',regex=True,inplace=True)
    pos_dat.loc[:,('start_next')]=(pos_dat.start.shift(-1))
    pos_dat.start_next.replace(np.nan,len(ten_k),inplace=True)
    pos_dat.loc[:,('end')]=pos_dat.start_next.astype('int')
    pos_dat.drop(['start_next'],axis=1,inplace=True)
    fin_dat=pos_dat.set_index('item').T.to_dict('list')
    
    item1a=ten_k[fin_dat['item1a'][0]:fin_dat['item1a'][1]]
    item7=ten_k[fin_dat['item7'][0]:fin_dat['item7'][1]]
    item7a=ten_k[fin_dat['item7a'][0]:fin_dat['item7a'][1]]
    
    return item1a, item7, item7a

In [ ]:
for _, ten_ks in ten_ks_by_ticker.items():
    for ten_k in ten_ks:
        item1a, item7, item7a = find_items(ten_k['file'])
        ten_k['item1a_raw'] = item1a
        ten_k['item7_raw'] = item7
        ten_k['item7a_raw'] = item7a


project_helper.print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['item1a_raw', 'item7_raw', 'item7a_raw'])

## Preprocess the Data
### Clean Up
As you can see, the text for the items is very messy. To clean this up, we'll remove the html and lowercase all the text.

In [ ]:
def remove_html_tags(text):
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    return text


def clean_text(text):
    text = text.lower()
    text = remove_html_tags(text)
    
    return text

Using the `clean_text` function, we'll clean up all the items.

In [ ]:
for _, ten_ks in ten_ks_by_ticker.items():
    for ten_k in ten_ks:
        ten_k['item1a'] = clean_text(ten_k['item1a_raw'])
        ten_k['item7'] = clean_text(ten_k['item7_raw'])
        ten_k['item7a'] = clean_text(ten_k['item7a_raw'])


project_helper.print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['item1a', 'item7', 'item7a'])

### Lemmatize
With the text cleaned up, it's time to distill the verbs down. Implement the `lemmatize_words` function to lemmitize verbs in the list of words provided.

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


def lemmatize_words(words):
    """
    Lemmatize words 

    Parameters
    ----------
    words : list of str
        List of words

    Returns
    -------
    lemmatized_words : list of str
        List of lemmatized words
    """
    
    # TODO: Implement
    
    return None


project_tests.test_lemmatize_words(lemmatize_words)

With the `lemmatize_words` function implemented, let's lemmatize all the item data.

In [ ]:
word_pattern = re.compile('\w+')


for _, ten_ks in ten_ks_by_ticker.items():
    for ten_k in ten_ks:
        ten_k['lemma_item1a'] = lemmatize_words(word_pattern.findall(ten_k['item1a']))
        ten_k['lemma_item7'] = lemmatize_words(word_pattern.findall(ten_k['item7']))  
        ten_k['lemma_item7a'] = lemmatize_words(word_pattern.findall(ten_k['item7a']))


project_helper.print_ten_k_data(ten_ks_by_ticker[example_ticker][:5], ['lemma_item1a', 'lemma_item7', 'lemma_item7a'])

# Analysis on 10ks
## Bag of Words
Let's extract some features from the text data using bag of words. Implement the `get_bag_of_words` using sklearn's `CountVectorizer` function.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


def get_bag_of_words(docs):
    """
    Generate a bag of words from documents

    Parameters
    ----------
    docs : list of str
        List of documents used to generate bag of words

    Returns
    -------
    bag_of_words : 2-d Numpy Ndarray
        Bag of words from the documents
    """
    
    # TODO: Implement
    
    return None


project_tests.test_get_bag_of_words(get_bag_of_words)

Time to generate the bag of words for each item corpus.

In [ ]:
bag_of_words_ten_ks = {}

for ticker, ten_ks in ten_ks_by_ticker.items():
    bag_of_words_ten_ks[ticker] = {
        'item1a': get_bag_of_words([' '.join(ten_k['lemma_item1a']) for ten_k in ten_ks]),
        'item7': get_bag_of_words([' '.join(ten_k['lemma_item7']) for ten_k in ten_ks]),
        'item7a': get_bag_of_words([' '.join(ten_k['lemma_item7a']) for ten_k in ten_ks])}


project_helper.print_ten_k_data([bag_of_words_ten_ks[example_ticker]], ['item1a', 'item7', 'item7a'])

## Jaccard Similarity
Using the bag of words, let's calculate the jaccard similarty and plot it over time.

In [ ]:
from sklearn.metrics import jaccard_similarity_score


def get_jaccard_similarity(docs):
    similarites = []
    for doc_i in range(len(docs) - 1):
        similarites.append(jaccard_similarity_score(docs[doc_i], docs[doc_i + 1]))
    
    return similarites


# Get dates for the universe
file_dates = {
    ticker: [ten_k['file_date'] for ten_k in ten_ks]
    for ticker, ten_ks in ten_ks_by_ticker.items()}  

item_1a_jaccard_similarities = get_jaccard_similarity(bag_of_words_ten_ks[example_ticker]['item1a'])
item_7_jaccard_similarities = get_jaccard_similarity(bag_of_words_ten_ks[example_ticker]['item7'])
item_7a_jaccard_similarities = get_jaccard_similarity(bag_of_words_ten_ks[example_ticker]['item7a'])


project_helper.plot_similarities(
    [item_1a_jaccard_similarities, item_7_jaccard_similarities, item_7a_jaccard_similarities],
    file_dates[example_ticker][1:],
    'Jaccard Similarities',
    ['item 1a', 'item 7', 'item 7a'])

## Word Frequency
Just like we did with the bag of words, lets create TFIDF values.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


def word_freq(docs):
    return TfidfVectorizer().fit_transform(docs).toarray()


word_freq_ten_ks = {}
for ticker, ten_ks in ten_ks_by_ticker.items():
    word_freq_ten_ks[ticker] = {
        'item1a': word_freq([' '.join(ten_k['lemma_item1a']) for ten_k in ten_ks]),
        'item7': word_freq([' '.join(ten_k['lemma_item7']) for ten_k in ten_ks]),
        'item7a': word_freq([' '.join(ten_k['lemma_item7a']) for ten_k in ten_ks])}


project_helper.print_ten_k_data([word_freq_ten_ks[example_ticker]], ['item1a', 'item7', 'item7a'])

## Cosine Similarity
Using the TFIDF values, we'll calculate the cosine similarity and plot it over time.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


def get_cosine_similarity(docs):
    return np.diag(cosine_similarity(docs, docs), k=1)


item_1a_cosine_similarities = get_cosine_similarity(word_freq_ten_ks[example_ticker]['item1a'])
item_7_cosine_similarities = get_cosine_similarity(word_freq_ten_ks[example_ticker]['item7'])
item_7a_cosine_similarities = get_cosine_similarity(word_freq_ten_ks[example_ticker]['item7a'])


project_helper.plot_similarities(
    [item_1a_cosine_similarities, item_7_cosine_similarities, item_7a_cosine_similarities],
    file_dates[example_ticker][1:],
    'Cosine Similarities',
    ['item 1a', 'item 7', 'item 7a'])

That's it! You've sucessfully done dentiment analysis on 10-ks!